In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd

In [2]:
"""model parameters"""
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)

model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-57.32
 Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, reward:-57.32
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, reward:-138.59
 Time: 1 | S: 12563951.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 1.0, Budget: 1000000.00, reward:-138.59
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, reward:-254.10
 Time: 2 | S: 12563922.0, E: 48.0, A: 0.0, I: 28.1, D: 0.0, R: 3.0, Budget: 1000000.00, reward:-254.10
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]


[0.16060925590883313, 0.16061397967007238]
[0.8137406886019865, 0.81374151974879]
[0.09733380419196237, 0.09733380569013378]
[0.043442195504156005, 0.04344220350287498]
 Time: 15 | S: 12483265.1, E: 5.1, A: 218.6, I: 389.5, D: 0.9, R: 80120.7, Budget: 853741.85, reward:-2988.66
 Time: 15 | S: 12483317.0, E: 4.0, A: 218.6, I: 389.5, D: 0.9, R: 80110.0, Budget: 853741.85, reward:-2988.66
 Time: 15 | new_e:2.4, cum_e:674.1, new_i:25.6, cum_i:807.6
[[2.1504787e-10 8.8006002e-01 6.8234653e-09 5.4474554e-06]
 [5.5143493e-01 1.7420597e-07 9.9899799e-01 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999928e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.4108756e-01 3.2196984e-02 9.9946576e-01 1.0000000e+00]]
[0.16061042330943573, 0.1606120699267292]
[0.8137413543673695, 0.8137415067409676]
[0.09733380535292212, 0.09733380552375986]
[0.04344220216412063, 0.04344220285060947]
 Time: 16 | S: 12476391.4, E: 4.4, A: 187.6, I: 385.6, D: 1.1, R: 87029.9, Budget: 845

[0.1606109134537563, 0.1606109148412716]
[0.8137415004149471, 0.8137415007179662]
[0.09733380551439702, 0.09733380551439705]
[0.04344220278577212, 0.043442202785772134]
 Time: 27 | S: 12310136.5, E: 0.5, A: 33.1, I: 157.7, D: 1.9, R: 253670.2, Budget: 692792.69, reward:-3068.70
 Time: 27 | S: 12310194.0, E: 0.0, A: 33.1, I: 157.7, D: 1.9, R: 253656.0, Budget: 692792.69, reward:-3068.70
 Time: 27 | new_e:0.3, cum_e:681.9, new_i:3.8, cum_i:932.8
[[1.3141532e-09 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [3.4955078e-01 1.4522536e-01 1.0000000e+00 1.0000000e+00]]
[0.16061091348238588, 0.16061091423821577]
[0.8137415004771859, 0.8137415006887431]
[0.09733380551439705, 0.09733380551439705]
[0.04344220278577213, 0.043442202785772134]
 Time: 28 | S: 12290043.9, E: 0.5, A: 28.3, I: 141.7, D: 1.9, R: 273783.6, Budget: 67

[0.16061091372701072, 0.1606109137302825]
[0.813741500607829, 0.8137415006191123]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 39 | S: 12080477.0, E: 0.2, A: 5.5, I: 41.5, D: 2.0, R: 483473.8, Budget: 523035.14, reward:-3085.98
 Time: 39 | S: 12080530.0, E: 0.0, A: 5.5, I: 41.5, D: 2.0, R: 483461.0, Budget: 523035.14, reward:-3085.98
 Time: 39 | new_e:0.1, cum_e:684.2, new_i:0.7, cum_i:952.7
[[0.00162733 0.9999993  0.9999945  0.99999917]
 [0.99999785 0.9999962  0.99999976 0.99999607]
 [0.99999475 0.99999654 0.9999912  0.9999994 ]
 [0.99999547 0.999995   0.9999958  0.9999925 ]
 [0.32102972 0.4603326  1.         1.        ]]
[0.16061091372729902, 0.1606109137295761]
[0.81374150060868, 0.8137415006186228]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 40 | S: 12062395.2, E: 0.2, A: 4.7, I: 37.0, D: 2.0, R: 501560.9, Budget: 508311.77, reward:-3086.62
 Time: 40 | S: 12062448.0, E: 0.0, A: 4.7,

[0.16061091372794675, 0.160610913727989]
[0.813741500613422, 0.8137415006158953]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 51 | S: 11874127.1, E: 0.0, A: 0.9, I: 10.0, D: 2.0, R: 689859.9, Budget: 335421.25, reward:-3089.45
 Time: 51 | S: 11874180.0, E: 0.0, A: 0.9, I: 10.0, D: 2.0, R: 689847.0, Budget: 335421.25, reward:-3089.45
 Time: 51 | new_e:0.0, cum_e:684.6, new_i:0.1, cum_i:956.2
[[2.05423145e-08 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.23943865e-01 7.00697541e-01 1.00000000e+00 1.00000000e+00]]
[0.16061091372795047, 0.16061091372797986]
[0.8137415006136085, 0.8137415006157881]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 52 | S: 11857911.5, E: 0.0, A: 0.8, I: 8.9, D: 2.0, R: 706076

[0.1606109137279588, 0.16061091372795935]
[0.813741500614648, 0.8137415006151902]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 63 | S: 11688774.2, E: 0.0, A: 0.1, I: 2.3, D: 2.1, R: 875221.4, Budget: 150967.82, reward:-3090.04
 Time: 63 | S: 11688828.0, E: 0.0, A: 0.1, I: 2.3, D: 2.1, R: 875214.0, Budget: 150967.82, reward:-3090.04
 Time: 63 | new_e:0.0, cum_e:684.6, new_i:0.0, cum_i:956.7
[[6.7171095e-07 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.8911447e-01 9.2614228e-01 1.0000000e+00 1.0000000e+00]]
[0.16061091372795885, 0.16061091372795924]
[0.8137415006146889, 0.8137415006151667]
[0.09733380551439705, 0.09733380551439705]
[0.043442202785772134, 0.043442202785772134]
 Time: 64 | S: 11674206.3, E: 0.0, A: 0.1, I: 2.0, D: 2.1, R: 889789.5, Budget: 136045.28

 Time: 75 | S: 11578832.9, E: 0.3, A: 0.0, I: 1.0, D: 2.1, R: 985163.7, Budget: 7167.70, reward:-3092.79
 Time: 75 | S: 11578891.0, E: 0.0, A: 0.0, I: 1.0, D: 2.1, R: 985156.0, Budget: 7167.70, reward:-3092.79
 Time: 75 | new_e:0.2, cum_e:685.2, new_i:0.2, cum_i:957.2
[[1.6612368e-08 9.9999642e-01 9.9999964e-01 9.9466527e-01]
 [9.9999964e-01 9.9999976e-01 9.9999988e-01 9.9999964e-01]
 [9.9999964e-01 9.9999964e-01 9.9999976e-01 9.9999988e-01]
 [9.9999976e-01 9.9999976e-01 9.9999964e-01 9.9999964e-01]
 [2.3337919e-02 9.9797732e-01 1.0000000e+00 1.0000000e+00]]
 Time: 76 | S: 11584305.7, E: 0.5, A: 0.0, I: 1.2, D: 2.1, R: 979690.6, Budget: 7167.70, reward:-3094.20
 Time: 76 | S: 11584363.0, E: 0.0, A: 0.0, I: 1.2, D: 2.1, R: 979684.0, Budget: 7167.70, reward:-3094.20
 Time: 76 | new_e:0.3, cum_e:685.5, new_i:0.3, cum_i:957.5
[[5.9338811e-07 9.9996281e-01 9.9995959e-01 3.2475388e-01]
 [9.9997747e-01 9.9997699e-01 9.9999392e-01 9.9997163e-01]
 [9.9996459e-01 9.9996710e-01 9.9998200e-01 9.99

In [6]:
(time.time() - tic) / ( T - start_time + 1)

47.90008162610671